In [ ]:
#Bruteforce
import itertools
import os

def brute_force_search(W, c, weights, values, class_index):
    n = len(weights) #total number of items in input file
    max_value = 0
    best_subset=[]
    
    for i in range(1, n+1):
        for subset in itertools.combinations(range(n), i):
            # Check if at least one item from each class is included
            if len(set(class_index[i] for i in subset)) == c:
                # Check if total weight is within limit
                if sum(weights[i] for i in subset) <= W:
                    # Calculate total value of the subset
                    subset_value = sum(values[i] for i in subset)
                    # Update max_value and best_subset if a better solution is found
                    if subset_value > max_value:
                        max_value = subset_value
                        best_subset.clear() #Reset the list of which items are placed in the knapsack
                        for i in range(n):
                            if i in subset:
                                best_subset.append(1)
                            else:
                                best_subset.append(0)
    if max_value==0:#There is no solution case
        for i in range(n):
            best_subset.append(0)
    return max_value, best_subset

#main
x=-1
while x!=0:
    temp= input("Please input the sequence number of the test case (if input x=0 the program will stop) \nx: ")
    if temp.isnumeric()==False:
        print("Error! You need to input a number\n")
        continue
    x=int(temp)
    if x==0:
        print("You have input x=0 so the program will stop")
        break
    in_path='INPUT_'+str(x)+'.txt'
    if os.path.isfile(in_path): # Check if input file exists
        with open(in_path, 'r') as fin: # Read input from file
            W = float(fin.readline().strip())
            c = int(fin.readline().strip())
            weights = list(map(float, fin.readline().strip().split(', ')))
            values = list(map(int, fin.readline().strip().split(', ')))
            class_index = list(map(int, fin.readline().strip().split(', ')))
            
        # Solve knapsack problem
        max_value, best_subset = brute_force_search(W, c, weights, values, class_index)
        
        # Write output to file
        out_path='OUTPUT_'+str(x)+'.txt'
        with open(out_path, 'w') as fout:
            fout.write(str(max_value) + '\n')
            fout.write(', '.join(str(i) for i in best_subset))
        
        print("Successful! Please check the output file.\n")
    else:
        print("Error! The input file does not exist.\n")

print("Thank you for using the program")

In [ ]:
#Branch And Bound
from typing import List, Tuple
import os





# These are the global variables used in this module. ----------------------------------------
BB_solution = None
BB_stack = []
BB_tree = []

# This is the structure used to represent a binary tree node. --------------------------------
class BTreeNode():
    def __init__(self):
        self.ParentNode = None
        self.Relaxation = 0
        self.Objective = 0
        self.ObjectID = -1
        self.Taken = None
        self.Room = None


# This solves the 0/1 Knapsack problem using the Branch and Bound method. --------------------
def BB_solver(capacity, weights, values, classes, numclasses):


    vtemp = values[:]


    # This is an array for indicating whether an element has been taken or not.
    global BB_tree, BB_stack

    items = len(values)     # Get number of items
    taken = [0] * items     # Allocate memory for taken
   


    arr = []

    for index, value in enumerate(values):
        arr.append((index, value/weights[index], classes[index])) 
  
    # This sorts the list in descending order.
    arr.sort(key=lambda pair:pair[1], reverse=True)
  
    for i in range(1, numclasses + 1):
        for j in arr:
            if i == j[2]:               
                values[j[0]]=9999999
                break





    value_per_weight = []

    for index, value in enumerate(values):
        value_per_weight.append((index, value/weights[index]))        



    # This sorts the list in descending order.
    value_per_weight.sort(key=lambda pair:pair[1], reverse=True)



    # This reorders the values and weights.
    weights = [weights[element[0]] for element in value_per_weight]
    values = [values[element[0]] for element in value_per_weight]
   



    # This creates a root node.
    Root = BTreeNode()
    Root.Room = capacity
    Root.Objective = 0
    Root.ObjectID = -1
    Root.Relaxation = getBound(items - 1, Root.ObjectID, Root.Room, Root.Objective, weights, values, value_per_weight)

    # This adds a root node to the tree.
    BB_tree.append(Root)
    BB_stack.append(Root)

    # This branches while the stack isn't empty.
    while BB_stack:
        Branch(items - 1, values, weights, value_per_weight)

    # This retraces which items were taken and which were ignored.
    Node = BB_solution

    while Node.ParentNode:
        taken[value_per_weight[Node.ObjectID][0]] = Node.Taken
        Node = Node.ParentNode

   

    maxvalue = 0
    for i in range(len(vtemp)):
         if taken[i] == 1:
             maxvalue += vtemp[i]
        
   

    return (maxvalue, taken)


# This handles all the branching logic and functionality. ------------------------------------
def Branch(items, values, weights, value_per_weight):

    global BB_solution, BB_tree, BB_stack

    if not BB_stack:
        return
    else:
        Root = BB_stack.pop()

    if BB_solution and Root.Relaxation < BB_solution.Objective:
        return
    elif Root.ObjectID == items:
        return

    Node = BTreeNode()
    Node.ObjectID = Root.ObjectID + 1
    Node.Room = Root.Room

    if Node.Room >= 0:
        Node.Objective = Root.Objective 
        Node.Taken = 0
        Node.Relaxation = getBound(items, Node.ObjectID, Node.Room, Node.Objective, weights, values, value_per_weight)
        Node.ParentNode = Root
        BB_stack.append(Node)

        if Node.Objective == Node.Relaxation:
            if BB_solution and Node.Objective > BB_solution.Objective:
                BB_solution = Node
            elif BB_solution is None:
                BB_solution = Node

    BB_tree.append(Node)

    Node = BTreeNode()
    Node.ObjectID = Root.ObjectID + 1
    Node.Room = Root.Room - weights[Node.ObjectID]
    
    if Node.Room >= 0:
        Node.Objective = Root.Objective + values[Node.ObjectID]
        Node.Taken = 1
        Node.Relaxation = \
            getBound(items, Node.ObjectID, Node.Room, \
            Node.Objective, weights, values, value_per_weight)
        Node.ParentNode = Root
        BB_stack.append(Node)

        if Node.Objective == Node.Relaxation:
            if BB_solution and Node.Objective > BB_solution.Objective:
                BB_solution = Node
            elif BB_solution is None:
                BB_solution = Node
    
    BB_tree.append(Node)

# This gets the bounds. ----------------------------------------------------------------------
def getBound(items, rootid, root_room, root_objective, weights, values, value_per_weight):
    while rootid < items and root_room - weights[rootid + 1] >= 0:
        root_objective = root_objective + values[rootid + 1]
        root_room = root_room - weights[rootid + 1]
        rootid = rootid + 1

    if rootid < items and root_room > 0:
        root_objective = root_objective + min(root_room, weights[rootid + 1]) * value_per_weight[rootid + 1][1] 
                  

    return root_objective



def load_input_file(input_file: str) -> Tuple[int, int, List[float], List[int], List[int]]:
    with open(input_file, 'r') as f:
        W = float(f.readline().strip())
        m = int(f.readline().strip())
        w = list(map(float, f.readline().strip().split(', ')))
        v = list(map(int, f.readline().strip().split(', ')))
        c = list(map(int, f.readline().strip().split(', ')))
    return W, m, w, v, c





x=-1
while x!=0:
    temp= input("Please input the sequence number of the test case (if input x=0 the program will stop) \nx: ")
    if temp.isnumeric()==False:
        print("Error! You need to input a number\n")
        continue
    x=int(temp)
    if x==0:
        print("You have input x=0 so the program will stop")
        break
    in_path='INPUT_'+str(x)+'.txt'
    if os.path.isfile(in_path): # Check if input file exists
         W, m, w, v, c = load_input_file(in_path)
         print("Data read:")
         print("Capacity: ",W)
         print("Number of classes: ",m)
         print("Weights:",w)
         print("values:",v)
            # Solve knapsack problem
         max_value, taken = BB_solver(W, w, v, c, m)

        # This outputs the final profit.
         print("--------------------------------------------------------------------------")
         print("Final profit: ", max_value)
         print(c)
         print(taken)
          # Write output to file
         out_path='OUTPUT_'+str(x)+'.txt'
         with open(out_path, 'w') as fout:
             fout.write(str(max_value) + '\n')
             fout.write(', '.join(str(i) for i in taken))
        
         print("Successful! Please check the output file.\n")

   
    else:
        print("Error! The input file does not exist.\n")

print("Thank you for using the program")



Best value: 322
Best combination: [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


# Mục mới

In [ ]:
# Internal Library
import random

def readInputFile(file_path):
    with open(file_path, 'r') as f:
        capacity = float(f.readline().strip())
        num_type = int(f.readline().strip())
        weights_str = f.readline().strip()
        weights = list(map(float, weights_str.split(',')))
        values_str = f.readline().strip()
        values = list(map(int, values_str.split(',')))
        types_str = f.readline().strip()
        types = list(map(int, types_str.split(',')))
    return capacity, num_type, weights, values, types

def writeOutputFile(file_path,best_value,solution):
    with open(file_path,'w') as g:
        g.write(str(best_value))
        g.write('\n')
        if solution != None: 
            n = len(solution)
            for i in range(n):
                g.write(str(solution[i])) 
                if(i != n-1): g.write(', ')
        else:
            g.write("None")

class External_Item:
    def __init__(self,weight,value,category):
        self.value = value
        self.weight = weight
        self.category = category

class Knapsack_Problem:
    def __init__(self,capacity,ex_items,category_num,population_size,gen_num,reproduction_rate=0.3,mutation_rate=0.2):
        self.capacity = capacity
        self.ex_items = ex_items
        self.population_size = population_size
        self.gen_num = gen_num
        self.category_num = category_num
        self.reproduction_rate = reproduction_rate
        self.mutation_rate = mutation_rate
        self.population = list()
    # Initial Population
    def generate_population(self):
        while len(self.population) <= self.population_size:
            genome = [random.choice([0,1]) for _ in range(len(self.ex_items))]
            if self.fitness_evaluation(genome) != 0:
                self.population.append(genome)
    # Evaluate The Fitness Of A Single Genome
    def fitness_evaluation(self,genome):
        if len(genome) != len(self.ex_items):
            raise ValueError("Genome and Items must be of the same length")
        total_value = 0
        total_weight = 0
        categories = set()
        for i,item in enumerate(self.ex_items):
            if genome[i] == 1:
                total_value += item.value
                total_weight += item.weight
                categories.add(item.category[i])
                if total_weight > self.capacity:
                    return 0
        if len(categories) < self.category_num:
            return 0
        return total_value
    # Tournament Selection To Get A Pair Of Parents
    def selection(self):
        random.shuffle(self.population)
        
        parent1 = list()
        parent2 = list()

        if self.fitness_evaluation(self.population[0]) > self.fitness_evaluation(self.population[1]):
            parent1 = self.population[0]
        else:
            parent1 = self.population[1]
        if self.fitness_evaluation(self.population[2]) > self.fitness_evaluation(self.population[3]):
            parent2 = self.population[2]
        else:
            parent2 = self.population[3]

        return parent1,parent2
    # Perform The Cross-Over Between 2 Genomes At A Random Point
    def cross_over(self,genome1,genome2):
        if len(genome1) != len(genome2):
            raise ValueError("Genomes 1 and 2 must be of same length")

        length = len(genome1)
        if length < 2:
            return genome1, genome2

        crossover_point = random.randint(1, length - 1)

        return genome1[0:crossover_point] + genome2[crossover_point:], genome2[0:crossover_point] + genome1[crossover_point:]
    # Flip A Random Bit In A Single Genome - Done
    def mutation(self,genome):
        mutate_bit = random.randint(0,len(self.ex_items)-1)
        if genome[mutate_bit] == 0:
            genome[mutate_bit] = 1
        else:
            genome[mutate_bit] = 0
        return genome
    # Choose Out One With The Best Fitness Value Among The Genomes In A Generation
    def get_best(self):
        fitness_values = list()

        for genome in self.population:
            fitness_values.append(self.fitness_evaluation(genome))

        max_fitness_value = max(fitness_values)
        max_fitness_index = fitness_values.index(max_fitness_value)

        return self.population[max_fitness_index]

    def solve(self):
        self.generate_population()
        for _ in range(self.gen_num):
            next_gen = []
            while len(next_gen) <= self.population_size:
                parent1, parent2 = self.selection()
                if random.uniform(0,1) < self.reproduction_rate:
                    child1,child2 = parent1, parent2
                else:
                    child1, child2 = self.cross_over(parent1,parent2)

                if random.uniform(0,1) < self.mutation_rate:
                    child1 = self.mutation(child1)
                if random.uniform(0,1) < self.mutation_rate:
                    child2 = self.mutation(child2)

                next_gen = next_gen + [child1,child2]

            self.population = next_gen
        return self.get_best()
# The Main Funtion Of The Program
if __name__ == "__main__":
    
    fileNum = input("Enter the number of the file name: ")
    inputFile = "INPUT_" + fileNum + ".txt"
    outputFile = "OUTPUT_" + fileNum + ".txt"

    weights = []
    values = []
    category = []
    capacity, category_num, weights, values, category = readInputFile(inputFile)
    
    items = [External_Item(weight,value,category) for weight,value in zip(weights,values)]

    knapsack = Knapsack_Problem(capacity,items,category_num,250,100)
    solution = knapsack.solve()
    best_value = 0
    for i,item in enumerate(items):
        if solution[i] == 1:
            best_value += item.value
    
    writeOutputFile(outputFile,best_value,solution)


Enter the number of the file name: 6


KeyboardInterrupt: ignored

In [ ]:
#local beam search
import time


def check_types(new_solution, types):

    # Check if each type appears at least once in the new solution.

    type_set = set()
    for k in range(len(types)):
        if new_solution[k] == 1:
            type_set.add(types[k])
    return len(type_set) == len(set(types))


def knapsack_local_beam(numType, weights, values, types, capacity, beam_width, max_iterations):

    # Solve the knapsack problem using the local beam search algorithm.

    n = len(weights)
    count = 1
    current_solutions = [([0] * n, 0)]

    for i in range(max_iterations):
        candidate_solutions = []
        for solution, value in current_solutions:
            for j in range(n):
                new_solution = solution[:]
                new_weight = float("inf")
                new_value = 0
                if count <= numType and types[j] == count:
                    new_solution[j] = 1 - new_solution[j]
                    new_value = sum(v * new_solution[k]
                                    for k, v in enumerate(values))
                    new_weight = sum(w * new_solution[k]
                                     for k, w in enumerate(weights))
                else:
                    if count > numType:
                        new_solution[j] = 1 - new_solution[j]
                        new_value = sum(v * new_solution[k]
                                        for k, v in enumerate(values))
                        new_weight = sum(
                            w * new_solution[k] for k, w in enumerate(weights))
                if new_weight <= capacity and (check_types(new_solution, types) or count < numType):
                    candidate_solutions.append((new_solution, new_value))
        count += 1
        candidate_solutions = sorted(
            candidate_solutions, key=lambda x: -x[1])[:beam_width]
        if candidate_solutions and candidate_solutions[0][1] > current_solutions[0][1]:
            current_solutions = candidate_solutions
        else:
            if count > numType:
                break
    solution, value = current_solutions[0]
    if check_types(solution, types):
        return value, solution
    else:
        return 0, None


def read_input_file(file_path):
    # Read the input file and return the necessary values.

    with open(file_path, "r") as f:
        capacity = float(f.readline().strip())
        num_type = int(f.readline().strip())
        weights = list(map(float, f.readline().strip().split(",")))
        values = list(map(float, f.readline().strip().split(",")))
        types = list(map(int, f.readline().strip().split(",")))
    return capacity, num_type, weights, values, types


def write_output_file(file_path, best_value, best_combination):

    # Write the output file with the best value and combination.

    with open(file_path, "w") as g:
        g.write(str(best_value))
        g.write("\n")
        if best_combination is not None:
            g.write(", ".join(str(c) for c in best_combination))
        else:
            g.write("None")


def minimum_list_types(numType, weights, types, capacity):

    # Check if it is possible to find a solution with at least one item of each type.

    min_list = [float("inf")] * numType
    for i in range(len(weights)):
        type_ = types[i] - 1
        if weights[i] < min_list[type_]:
            min_list[type_] = weights[i]
    return sum(min_list) < capacity

# driver function


def main(inputFile, outputFile):
    capacity = 0
    numType = 0
    weights = []
    values = []
    types = []
    capacity, numType, weights, values, types = read_input_file(inputFile)
    beam_width = 1000
    max_iterations = 10000
    best_value = 0
    best_combination = None
    if minimum_list_types(numType, weights, types, capacity) == True:
        best_value, best_combination = knapsack_local_beam(
            numType, weights, values, types, capacity, beam_width, max_iterations)
    else:
        print("None")
    write_output_file(outputFile, best_value, best_combination)
    print("Best value:", best_value)
    print("Best combination:", best_combination)


if __name__ == "__main__":
    testcase = input('Enter test case: ')
    filename = f"INPUT_{testcase}.txt"
    outputFile = f"OUTPUT_{testcase}.txt"

    inputFile = filename

    startTime = time.time()
    main(inputFile, outputFile)
    endTime = time.time()
    print("Runtime: ", endTime-startTime, "seconds")
